In [1]:
# --- CODE CELL 1: Imports + Config + Functions (functions first) ---

import os
import re
import json
import time
import datetime as dt
from pathlib import Path
from typing import Dict, Any, Optional, Tuple, List

import requests
import pandas as pd

In [2]:
# =========================
# Config
# =========================

ALPHAVANTAGE_API_KEY = "OYA0CFIEESAINF1M"
AV_BASE_URL = "https://www.alphavantage.co/query"

DATA_DIR = Path("data")
SECURITY_MASTER_DIR = DATA_DIR / "security_master"
ETF_HOLDINGS_DIR = DATA_DIR / "etf_holdings"

SECURITY_MASTER_PARQUET = SECURITY_MASTER_DIR / "security_master.parquet"
SECURITY_MASTER_META = SECURITY_MASTER_DIR / "meta.json"

In [3]:
# =========================
# Helpers
# =========================

def ensure_dirs():
    SECURITY_MASTER_DIR.mkdir(parents=True, exist_ok=True)
    ETF_HOLDINGS_DIR.mkdir(parents=True, exist_ok=True)

def now_ts() -> str:
    return dt.datetime.now().strftime("%Y%m%d_%H%M%S")

def today_str() -> str:
    return dt.date.today().isoformat()

def symbol_norm(symbol: str) -> str:
    """
    Normalize ticker symbols for joining across sources.

    Rules (simple + practical):
    - uppercase, strip
    - replace '.' and '/' with '-'
    - collapse consecutive '-' into one
    """
    if symbol is None:
        return ""
    s = str(symbol).upper().strip()
    s = s.replace(".", "-").replace("/", "-")
    s = re.sub(r"-{2,}", "-", s)
    return s

def safe_float(x) -> Optional[float]:
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    try:
        if isinstance(x, str):
            x = x.replace("$", "").replace(",", "").strip()
        return float(x)
    except Exception:
        return None

def parse_market_cap(x) -> Optional[float]:
    # Nasdaq screener often already gives a raw integer. Sometimes blank.
    return safe_float(x)

def parse_last_sale(x) -> Optional[float]:
    return safe_float(x)

def alpha_vantage_get(params: Dict[str, Any], timeout: int = 30) -> Dict[str, Any]:
    """
    Alpha Vantage GET wrapper.
    Notes:
    - AV sometimes returns "Note" (rate limit) or "Information" (errors).
    """
    params = dict(params)
    params["apikey"] = ALPHAVANTAGE_API_KEY
    r = requests.get(AV_BASE_URL, params=params, timeout=timeout)
    r.raise_for_status()
    data = r.json()

    # Rate limit / errors
    if isinstance(data, dict) and ("Note" in data or "Information" in data or "Error Message" in data):
        raise RuntimeError(f"Alpha Vantage response indicates an issue: {data}")

    return data


# =========================
# Security Master (Nasdaq Screener CSV)
# =========================

REQUIRED_MASTER_COLS = {"Symbol", "Name", "Country", "Sector", "Industry"}

def load_security_master() -> pd.DataFrame:
    ensure_dirs()
    if SECURITY_MASTER_PARQUET.exists():
        df = pd.read_parquet(SECURITY_MASTER_PARQUET)
        # Ensure symbol_norm exists
        if "symbol_norm" not in df.columns and "Symbol" in df.columns:
            df["symbol_norm"] = df["Symbol"].map(symbol_norm)
        return df
    # empty shell
    return pd.DataFrame(columns=["Symbol","Name","Country","Sector","Industry","symbol_norm"])

def refresh_security_master_from_csv(csv_path: str) -> pd.DataFrame:
    """
    Reads the Nasdaq Stock Screener CSV and persists a cleaned version as parquet.
    Keeps the raw CSV with timestamp so users can roll back if needed.
    """
    ensure_dirs()
    raw_path = Path(csv_path)
    if not raw_path.exists():
        raise FileNotFoundError(f"CSV not found at: {csv_path}")

    # Save raw copy for audit/rollback
    raw_copy = SECURITY_MASTER_DIR / f"raw_{now_ts()}.csv"
    raw_copy.write_bytes(raw_path.read_bytes())

    df = pd.read_csv(raw_copy)

    missing = REQUIRED_MASTER_COLS - set(df.columns)
    if missing:
        raise ValueError(f"Security master CSV is missing required columns: {sorted(list(missing))}")

    # Keep a minimal set, but you can store more if you want
    keep_cols = [c for c in ["Symbol","Name","Last Sale","Market Cap","Country","IPO Year","Sector","Industry"] if c in df.columns]
    df = df[keep_cols].copy()

    df["Symbol"] = df["Symbol"].astype(str).str.strip()
    df["symbol_norm"] = df["Symbol"].map(symbol_norm)

    # Optional cleaning of numeric columns
    if "Last Sale" in df.columns:
        df["last_sale_num"] = df["Last Sale"].map(parse_last_sale)
    if "Market Cap" in df.columns:
        df["market_cap_num"] = df["Market Cap"].map(parse_market_cap)

    # Drop rows with empty symbols
    df = df[df["symbol_norm"] != ""].copy()

    # Handle duplicates: keep the row with max market cap if available; else first
    if "market_cap_num" in df.columns:
        df = df.sort_values(["symbol_norm","market_cap_num"], ascending=[True, False])
    df = df.drop_duplicates(subset=["symbol_norm"], keep="first")

    # Persist parquet + meta
    df.to_parquet(SECURITY_MASTER_PARQUET, index=False)

    meta = {
        "uploaded_at": dt.datetime.now().isoformat(),
        "raw_copy": str(raw_copy),
        "row_count": int(len(df)),
        "columns": list(df.columns),
    }
    SECURITY_MASTER_META.write_text(json.dumps(meta, indent=2))

    return df


# =========================
# ETF Holdings Cache (Alpha Vantage ETF_PROFILE)
# =========================

def holdings_cache_path(etf_symbol: str, asof_date: Optional[str] = None) -> Path:
    ensure_dirs()
    etf = symbol_norm(etf_symbol)
    d = asof_date or today_str()
    folder = ETF_HOLDINGS_DIR / etf
    folder.mkdir(parents=True, exist_ok=True)
    return folder / f"holdings_{d}.parquet"

def load_cached_etf_holdings(etf_symbol: str, asof_date: Optional[str] = None) -> Optional[pd.DataFrame]:
    path = holdings_cache_path(etf_symbol, asof_date)
    if path.exists():
        return pd.read_parquet(path)
    return None

def refresh_etf_holdings(etf_symbol: str, asof_date: Optional[str] = None, sleep_seconds: float = 0.0) -> pd.DataFrame:
    """
    Fetches ETF holdings from Alpha Vantage ETF_PROFILE and caches a daily snapshot.
    """
    etf_symbol = symbol_norm(etf_symbol)
    d = asof_date or today_str()

    data = alpha_vantage_get({"function": "ETF_PROFILE", "symbol": etf_symbol})

    # The exact JSON structure can vary. We handle a few common patterns:
    # - data may include a "holdings" array
    # - or might embed holdings under a key like "holdings" / "constituents"
    holdings = None
    for k in ["holdings", "Holdings", "constituents", "Constituents"]:
        if k in data and isinstance(data[k], list):
            holdings = data[k]
            break

    if holdings is None:
        # Last resort: try to find a list-of-dicts within the response that looks like holdings
        for v in data.values():
            if isinstance(v, list) and len(v) > 0 and isinstance(v[0], dict):
                # heuristic: must contain a symbol-like field and a weight/allocation-like field
                keys = set(v[0].keys())
                if any(k.lower() in keys for k in ["symbol","ticker","holding","asset"]) and any(k.lower() in keys for k in ["weight","allocation","percentage"]):
                    holdings = v
                    break

    if holdings is None:
        raise ValueError(f"Could not locate holdings list in ETF_PROFILE response for {etf_symbol}. Keys: {list(data.keys())}")

    hdf = pd.DataFrame(holdings).copy()

    # Normalize columns we care about:
    # Try to infer constituent symbol and weight columns.
    col_map = {}
    lower_cols = {c: c.lower() for c in hdf.columns}

    # Symbol column candidates
    sym_candidates = [c for c in hdf.columns if lower_cols[c] in ["symbol","ticker","holding","asset","constituent"]]
    # Weight candidates
    wt_candidates = [c for c in hdf.columns if any(x in lower_cols[c] for x in ["weight","allocation","percentage","pct"])]

    if not sym_candidates:
        # fallback: pick first column with "sym" in name
        sym_candidates = [c for c in hdf.columns if "sym" in lower_cols[c]]
    if not wt_candidates:
        wt_candidates = [c for c in hdf.columns if "weight" in lower_cols[c] or "alloc" in lower_cols[c]]

    if not sym_candidates or not wt_candidates:
        raise ValueError(f"Could not infer holdings symbol/weight columns for {etf_symbol}. Columns: {list(hdf.columns)}")

    sym_col = sym_candidates[0]
    wt_col = wt_candidates[0]

    out = pd.DataFrame({
        "etf_symbol": etf_symbol,
        "constituent_symbol_raw": hdf[sym_col].astype(str).str.strip(),
        "constituent_symbol_norm": hdf[sym_col].astype(str).map(symbol_norm),
        "weight_raw": hdf[wt_col],
    })

    # Convert weights into decimal (0-1). If already decimals, keep. If % (0-100), convert.
    out["weight"] = out["weight_raw"].map(safe_float)
    if out["weight"].notna().any():
        # If typical weight > 1, assume it is percent points
        if out["weight"].dropna().median() > 1.0:
            out["weight"] = out["weight"] / 100.0

    out = out.dropna(subset=["constituent_symbol_norm","weight"])
    out = out[out["constituent_symbol_norm"] != ""].copy()

    # Cache
    path = holdings_cache_path(etf_symbol, d)
    out.to_parquet(path, index=False)

    if sleep_seconds > 0:
        time.sleep(sleep_seconds)

    return out


# =========================
# Portfolio normalization + look-through
# =========================

def normalize_portfolio_inputs(df: pd.DataFrame, total_portfolio_value: Optional[float] = None) -> pd.DataFrame:
    """
    Supported inputs:
      - ticker (required)
      - percent (optional; 0..1)
      - shares (optional)
      - price_per_share (optional)
      - dollars (optional)

    We compute position_value:
      - dollars if provided
      - else shares * price_per_share if available
      - else percent * total_portfolio_value if provided
    """
    df = df.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    if "ticker" not in df.columns:
        raise ValueError("Portfolio input must contain a 'ticker' column.")

    df["ticker_raw"] = df["ticker"].astype(str)
    df["ticker_norm"] = df["ticker_raw"].map(symbol_norm)

    if "price_per_share" not in df.columns:
        df["price_per_share"] = pd.NA

    def compute_value(row):
        if pd.notna(row.get("dollars")):
            return float(row["dollars"])
        if pd.notna(row.get("shares")) and pd.notna(row.get("price_per_share")):
            return float(row["shares"]) * float(row["price_per_share"])
        if pd.notna(row.get("percent")):
            if total_portfolio_value is None:
                raise ValueError("total_portfolio_value must be provided if any row uses 'percent'.")
            return float(row["percent"]) * float(total_portfolio_value)
        return None

    df["position_value"] = df.apply(compute_value, axis=1)

    bad = df[df["position_value"].isna()]
    if len(bad) > 0:
        raise ValueError(
            "Some rows are missing enough info to compute dollars exposure. "
            "Provide dollars OR shares+price_per_share OR percent+total_portfolio_value.\n"
            f"{bad[['ticker_raw','percent','shares','price_per_share','dollars']].to_string(index=False)}"
        )

    return df[["ticker_raw","ticker_norm","percent","shares","price_per_share","dollars","position_value"]]

def classify_assets_with_master(portfolio_df: pd.DataFrame, master_df: pd.DataFrame) -> pd.DataFrame:
    """
    Without a dedicated instrument-type API, we use a practical approach:
    - If ticker appears in the security master, it's likely an equity or ETF (both can appear).
    - We allow a user override column 'is_etf' if present; otherwise default to False.
      (In a real app, you'd have a small override UI for this.)
    """
    df = portfolio_df.copy()
    if "is_etf" not in df.columns:
        df["is_etf"] = False  # user can override later
    df["asset_type"] = df["is_etf"].map(lambda x: "ETF" if bool(x) else "Stock")

    # Enrich direct holdings from master (optional)
    master_cols = ["symbol_norm","Name","Country","Sector","Industry"]
    master_min = master_df[master_cols].drop_duplicates("symbol_norm") if len(master_df) else pd.DataFrame(columns=master_cols)
    df = df.merge(master_min, left_on="ticker_norm", right_on="symbol_norm", how="left")
    df = df.drop(columns=["symbol_norm"])
    return df

def build_lookthrough_exposures(
    portfolio_df: pd.DataFrame,
    master_df: pd.DataFrame,
    refresh_missing_etf_holdings: bool = True,
    asof_date: Optional[str] = None
) -> pd.DataFrame:
    """
    Returns a unified exposures table:
      - source_ticker_norm, source_type
      - underlying_symbol_norm
      - exposure_value
      - plus Name/Country/Sector/Industry from the local master for underlyings
    """
    rows = []
    d = asof_date or today_str()

    for _, r in portfolio_df.iterrows():
        src = r["ticker_norm"]
        src_type = r["asset_type"]
        pv = float(r["position_value"])

        if src_type == "Stock":
            rows.append({
                "source_ticker_norm": src,
                "source_type": src_type,
                "underlying_symbol_norm": src,
                "exposure_value": pv,
            })
        else:
            cached = load_cached_etf_holdings(src, d)
            if cached is None and refresh_missing_etf_holdings:
                cached = refresh_etf_holdings(src, d)

            if cached is None or len(cached) == 0:
                rows.append({
                    "source_ticker_norm": src,
                    "source_type": src_type,
                    "underlying_symbol_norm": None,
                    "exposure_value": pv,
                })
            else:
                for _, h in cached.iterrows():
                    rows.append({
                        "source_ticker_norm": src,
                        "source_type": src_type,
                        "underlying_symbol_norm": h["constituent_symbol_norm"],
                        "exposure_value": pv * float(h["weight"]),
                    })

    exp = pd.DataFrame(rows)

    # Join to local security master for slicing
    master_cols = ["symbol_norm","Name","Country","Sector","Industry"]
    master_min = master_df[master_cols].drop_duplicates("symbol_norm") if len(master_df) else pd.DataFrame(columns=master_cols)

    exp = exp.merge(master_min, left_on="underlying_symbol_norm", right_on="symbol_norm", how="left")
    exp = exp.drop(columns=["symbol_norm"])

    # A couple convenience columns
    exp["company_name"] = exp["Name"]
    exp["country"] = exp["Country"]
    exp["sector"] = exp["Sector"]
    exp["industry"] = exp["Industry"]

    return exp

def build_slices(exposures: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      - by_company
      - by_sector
      - by_country
      - by_source_vehicle
    """
    by_company = (
        exposures.dropna(subset=["underlying_symbol_norm"])
        .groupby(["underlying_symbol_norm","company_name"], dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
    )

    by_sector = (
        exposures.groupby("sector", dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
    )

    by_country = (
        exposures.groupby("country", dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
    )

    by_source_vehicle = (
        exposures.groupby(["source_ticker_norm","source_type"], dropna=False)
        .agg(total_exposure=("exposure_value","sum"))
        .sort_values("total_exposure", ascending=False)
    )

    return by_company, by_sector, by_country, by_source_vehicle

In [4]:
# --- CODE CELL 2: Demo Run (Upload/Refresh Master + Portfolio + Look-through) ---

ensure_dirs()

# 1) USER ACTION: refresh the local security master whenever they want
# Replace this with the path to the CSV downloaded from "Nasdaq Stock Screener"
# Example: "downloads/nasdaq_screener.csv"
SECURITY_MASTER_CSV_PATH = "nasdaq_screener.csv"

try:
    master = refresh_security_master_from_csv(SECURITY_MASTER_CSV_PATH)
    print(f"Security master refreshed. Rows: {len(master)}")
except FileNotFoundError:
    master = load_security_master()
    print("No security master CSV found yet. Loaded existing cached master (may be empty).")
    if len(master) == 0:
        print("-> Put your screener CSV at SECURITY_MASTER_CSV_PATH and rerun this cell.")

display(master.head(5))

Security master refreshed. Rows: 6989


,Symbol,Name,Last Sale,Market Cap,Country,IPO Year,Sector,Industry,symbol_norm,last_sale_num,market_cap_num
0,A,Agilent Technologies Inc. Common Stock,$138.39,3.923341e+10,United States,1999.0,Industrials,Biotechnology: Laboratory Analytical Instruments,A,138.390,3.923341e+10
1,AA,Alcoa Corporation Common Stock,$54.25,1.404880e+10,United States,2016.0,Industrials,Aluminum,AA,54.250,1.404880e+10
2,AACB,Artius II Acquisition Inc. Class A Ordinary Sh...,$10.29,0.000000e+00,United States,2025.0,NaN,NaN,AACB,10.290,0.000000e+00
3,AACBR,Artius II Acquisition Inc. Rights,$0.33,0.000000e+00,United States,2025.0,NaN,NaN,AACBR,0.330,0.000000e+00
4,AACG,ATA Creativity Global American Depositary Shares,$0.744,2.374207e+07,China,2008.0,Real Estate,Other Consumer Services,AACG,0.744,2.374207e+07


In [5]:
# 2) Portfolio input (user-provided). For the POC, we require enough info to compute $ exposure.
# If you don't want to call any price APIs, just ask users to provide dollars or shares+price_per_share.
portfolio_input = pd.DataFrame([
    {"ticker": "AAPL", "shares": 20, "price_per_share": 190},      # direct stock
    {"ticker": "MSFT", "dollars": 3000},                           # direct stock
    {"ticker": "QQQ", "percent": 0.40, "is_etf": True},            # ETF (needs total value)
])

TOTAL_PORTFOLIO_VALUE = 20000  # required if any row uses percent

portfolio_norm = normalize_portfolio_inputs(portfolio_input, total_portfolio_value=TOTAL_PORTFOLIO_VALUE)

# Bring forward any user-provided is_etf override
if "is_etf" in portfolio_input.columns:
    overrides = portfolio_input.copy()
    overrides["ticker_norm"] = overrides["ticker"].map(symbol_norm)
    overrides = overrides[["ticker_norm","is_etf"]].dropna()
    portfolio_norm = portfolio_norm.merge(overrides, on="ticker_norm", how="left")
    portfolio_norm["is_etf"] = portfolio_norm["is_etf"].fillna(False)
else:
    portfolio_norm["is_etf"] = False

portfolio = classify_assets_with_master(portfolio_norm, master)

print("\nNormalized portfolio:")
display(portfolio)


Normalized portfolio:


/var/folders/mx/n9j2tnf56sz57xdgr922njrw0000gn/T/ipykernel_29083/1737436676.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  portfolio_norm["is_etf"] = portfolio_norm["is_etf"].fillna(False)


,ticker_raw,ticker_norm,percent,shares,price_per_share,dollars,position_value,is_etf,asset_type,Name,Country,Sector,Industry
0,AAPL,AAPL,NaN,20.0,190.0,NaN,3800.0,False,Stock,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing
1,MSFT,MSFT,NaN,NaN,NaN,3000.0,3000.0,False,Stock,Microsoft Corporation Common Stock,United States,Technology,Computer Software: Prepackaged Software
2,QQQ,QQQ,0.4,NaN,NaN,NaN,8000.0,True,ETF,NaN,NaN,NaN,NaN


In [6]:
# 2) Portfolio input (user-provided). For the POC, we require enough info to compute $ exposure.
# If you don't want to call any price APIs, just ask users to provide dollars or shares+price_per_share.
portfolio_input = pd.DataFrame([
    {"ticker": "AAPL", "shares": 20, "price_per_share": 190},      # direct stock
    {"ticker": "MSFT", "dollars": 3000},                           # direct stock
    {"ticker": "QQQ", "percent": 0.40, "is_etf": True},            # ETF (needs total value)
])

TOTAL_PORTFOLIO_VALUE = 20000  # required if any row uses percent

portfolio_norm = normalize_portfolio_inputs(portfolio_input, total_portfolio_value=TOTAL_PORTFOLIO_VALUE)

# Bring forward any user-provided is_etf override
if "is_etf" in portfolio_input.columns:
    overrides = portfolio_input.copy()
    overrides["ticker_norm"] = overrides["ticker"].map(symbol_norm)
    overrides = overrides[["ticker_norm","is_etf"]].dropna()
    portfolio_norm = portfolio_norm.merge(overrides, on="ticker_norm", how="left")
    portfolio_norm["is_etf"] = portfolio_norm["is_etf"].fillna(False)
else:
    portfolio_norm["is_etf"] = False

portfolio = classify_assets_with_master(portfolio_norm, master)

print("\nNormalized portfolio:")
display(portfolio)


Normalized portfolio:


/var/folders/mx/n9j2tnf56sz57xdgr922njrw0000gn/T/ipykernel_29083/1737436676.py:19: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  portfolio_norm["is_etf"] = portfolio_norm["is_etf"].fillna(False)


,ticker_raw,ticker_norm,percent,shares,price_per_share,dollars,position_value,is_etf,asset_type,Name,Country,Sector,Industry
0,AAPL,AAPL,NaN,20.0,190.0,NaN,3800.0,False,Stock,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing
1,MSFT,MSFT,NaN,NaN,NaN,3000.0,3000.0,False,Stock,Microsoft Corporation Common Stock,United States,Technology,Computer Software: Prepackaged Software
2,QQQ,QQQ,0.4,NaN,NaN,NaN,8000.0,True,ETF,NaN,NaN,NaN,NaN


In [7]:
# 3) Build look-through exposures
# This will try to load cached ETF holdings for today; if missing, it will call Alpha Vantage once per ETF.
# IMPORTANT: Alpha Vantage free tier is rate limited; caching avoids repeated calls.
exposures = build_lookthrough_exposures(
    portfolio_df=portfolio,
    master_df=master,
    refresh_missing_etf_holdings=True,
    asof_date=today_str()
)

print("\nUnified exposures (top 25):")
display(exposures.sort_values("exposure_value", ascending=False).head(25))

# 4) Slices
by_company, by_sector, by_country, by_source_vehicle = build_slices(exposures)

print("\nExposure by Company (top 25):")
display(by_company.head(25))

print("\nExposure by Sector:")
display(by_sector)

print("\nExposure by Country:")
display(by_country)

print("\nExposure by Source Vehicle (Direct vs ETF):")
display(by_source_vehicle)


Unified exposures (top 25):


,source_ticker_norm,source_type,underlying_symbol_norm,exposure_value,Name,Country,Sector,Industry,company_name,country,sector,industry
0,AAPL,Stock,AAPL,3800.0,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing
1,MSFT,Stock,MSFT,3000.0,Microsoft Corporation Common Stock,United States,Technology,Computer Software: Prepackaged Software,Microsoft Corporation Common Stock,United States,Technology,Computer Software: Prepackaged Software
2,QQQ,ETF,NVDA,719.2,NVIDIA Corporation Common Stock,United States,Technology,Semiconductors,NVIDIA Corporation Common Stock,United States,Technology,Semiconductors
3,QQQ,ETF,AAPL,635.2,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing,Apple Inc. Common Stock,United States,Technology,Computer Manufacturing
4,QQQ,ETF,MSFT,569.6,Microsoft Corporation Common Stock,United States,Technology,Computer Software: Prepackaged Software,Microsoft Corporation Common Stock,United States,Technology,Computer Software: Prepackaged Software
5,QQQ,ETF,AMZN,389.6,Amazon.com Inc. Common Stock,United States,Consumer Discretionary,Catalog/Specialty Distribution,Amazon.com Inc. Common Stock,United States,Consumer Discretionary,Catalog/Specialty Distribution
6,QQQ,ETF,TSLA,336.8,Tesla Inc. Common Stock,United States,Industrials,Auto Manufacturing,Tesla Inc. Common Stock,United States,Industrials,Auto Manufacturing
7,QQQ,ETF,META,308.0,Meta Platforms Inc. Class A Common Stock,United States,Technology,Computer Software: Programming Data Processing,Meta Platforms Inc. Class A Common Stock,United States,Technology,Computer Software: Programming Data Processing
8,QQQ,ETF,GOOGL,286.4,Alphabet Inc. Class A Common Stock,United States,Technology,Computer Software: Programming Data Processing,Alphabet Inc. Class A Common Stock,United States,Technology,Computer Software: Programming Data Processing
9,QQQ,ETF,GOOG,268.0,Alphabet Inc. Class C Capital Stock,United States,Technology,Computer Software: Programming Data Processing,Alphabet Inc. Class C Capital Stock,United States,Technology,Computer Software: Programming Data Processing



Exposure by Company (top 25):


,,total_exposure
underlying_symbol_norm,company_name,
AAPL,Apple Inc. Common Stock,4435.2
MSFT,Microsoft Corporation Common Stock,3569.6
NVDA,NVIDIA Corporation Common Stock,719.2
AMZN,Amazon.com Inc. Common Stock,389.6
TSLA,Tesla Inc. Common Stock,336.8
META,Meta Platforms Inc. Class A Common Stock,308.0
GOOGL,Alphabet Inc. Class A Common Stock,286.4
GOOG,Alphabet Inc. Class C Capital Stock,268.0
AVGO,Broadcom Inc. Common Stock,259.2



Exposure by Sector:


,total_exposure
sector,
Technology,11833.6
Consumer Discretionary,1148.8
Industrials,589.6
Health Care,434.4
Telecommunications,289.6
Consumer Staples,195.2
Utilities,114.4
Basic Materials,85.6
Real Estate,56.0



Exposure by Country:


,total_exposure
country,
United States,14447.2
Canada,89.6
Netherlands,65.6
United Kingdom,48.0
Argentina,44.0
NaN,36.0
China,32.8
Ireland,26.4
Australia,12.0



Exposure by Source Vehicle (Direct vs ETF):


,,total_exposure
source_ticker_norm,source_type,
QQQ,ETF,8001.6
AAPL,Stock,3800.0
MSFT,Stock,3000.0
